# 1. Colab Setting

In [2]:
# Set the directory path where training data and fine-tuned model files exist
path = "/content/drive/MyDrive/Solution Challenge/Algorithm/"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680543 sha256=599a77eb628e1e85f977a37add35d177634c6a92797a2b59c2f3fe99633bef7d
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9ixgye67
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9ixgye67
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
# KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
# Using GPU
device = torch.device("cuda:0")

In [8]:
# BERT model, load vocabulary
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2. Dataset Pre-processing

In [9]:
import pandas as pd
conversation_data = pd.read_excel(path + "conversation_data_set.xlsx")

In [10]:
data_list = []
for q, label in zip(conversation_data.iloc[:,1], conversation_data.iloc[:,2])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

In [11]:
# Check few sentences for train and test
print(data_list[0])
print(data_list[400])
print(data_list[800])
print(data_list[1200])
print(data_list[1600])
print(data_list[2000])
print(data_list[-1])

['여보세요 응 이런 사람은 알고 계세요. 건물이나 돈을 범죄 사겠다는 권고하는 과정에서 대량의 신용카드를 대포통령이 발견이 됐는데 그중 문제가 되는게 거기서 지금 명의로 된 하나은행 통장과 농협 통장에 발견이 됐어요. 그 두 개의 통장 개설에 대해서 아시는 내용 있으세요 아니요? 전혀 없어요음 그 두 개의 통장을 작년 3월 3일 경기도 광명시 철산동에서 바둑 계절을 개설된 계좌인데 어 전혀 모르셨어요음네 알겠습니다. 끝없이 통화 불편하시면 말씀을 해 주세요음 제출 계좌를 불법 현장에서 알 수 있기 때문에 동결 처리와 동시에 북구로 환수 조치했고요. 금융 운동은 제주공 결과 또 계좌번호도 확실히 명의 계정이한테 저희가 확인을 했어요. 제가 어 저에게 연락드린 이유는 어? 본인이 직접 통장을 개설하셔서 일당들에게 금전적인 대가를 받고 통장을 판매하신 건 아닌지 일단 아니면은 제 개인 정보가 유출이 되면서 명의 조정 당하실 피해자 분이신지에 대해서 저는 연락을 드린 거예요. 통장을 판매는 양봉하신 적 있으세요 아니요? 통장을 판매를 양도 하시는 것도 아니라고 보이는 하셨고도 모른다고 보기는 하셨는데 그러면은 사이에 지갑이나 핸드폰이랑 개인정보가 있을 만한 물건을 분실하신 데가 있을까요 아니요 없어요음 질문을 좀 똑바로 좀 해주세요 장난치지 마시고. 장난 아닌데요. 네, 알겠습니다. 자 그러면은 아 혹시 이런 쇼핑몰 사이트를 이용하세요 11번가나 옥션 지마켓. 라면 그냥 저희가 이걸 왜 못 잡았냐면 대부분 평소들이 이런 식품으로 사이트에서 개인 정보였지 상당히 많이 됐더라고요. 안녕! 네 저 지금 이거 조사를 어 한 두 시간 정도 할 건데 여기 괜찮으세요? 자.', '1']
['서울중앙지방검찰청에서 전화드린 거예요네네 자 현재 저희 검찰에서 김용식 주범으로 인한 금융범죄 사기단의 검거한 상태예요. 자이 검거 현장에서 대량의 신용카드가 대포통장을 압수하던 과정에 연기로 돼 이제 농협 하나는 통장 두 점이 발견되어 조석제 연락드린 거고요. 자 문의 혹시이 통장 개별 거

# 3. Split Data into Train and Test

In [12]:
# Split data into train and test
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [13]:
# Check the number of sentences
print(len(dataset_train))
print(len(dataset_test))

1645
549


# 4. Prepare KoBERT and Input Data

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
# Tokenization
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5. Prepare KoBERT classifier

In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # Modify the number of classes
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
# Load KoBERT model
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Set optimizer and schedule
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# Define a function for accuracy measure
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6. Train KoBERT Model

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# Save the trained pt file to target path
torch.save(model, path + "conversation_model.pt")

# 7. Test New Sentences

In [20]:
import numpy as np

In [21]:
# Load trained pt file
model_conv = torch.load("/content/drive/MyDrive/KAIST/2022winter/voice_phishing/conversation_model_res.pt")

In [22]:
# Tokenization
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
def softmax(a):
    a = a / 2
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model_conv.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model_conv(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            if np.argmax(logits) == 0:
                test_eval.append("일반 대화")
            elif np.argmax(logits) == 1:
                test_eval.append("보이스피싱")
            result = softmax(logits)
            
    result_list = result.tolist()
    print(f"{predict_sentence}")
    print(f">> scam call probability: {result_list[-1]*100}% \n")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [23]:
sentence = "현장에서 대포 통장과 신용카드가 발견되었는데 그 중에서 본인 명의로 통장이 발견돼서 연락을 드렸거든요."
print(predict(sentence))

sentence = "최근에 길고양이가 유기 센터에 들어오게 돼서 30일도 안 된 아기고양이를 우리 집에 데려온 적이 있어."
print(predict(sentence))

현장에서 대포 통장과 신용카드가 발견되었는데 그 중에서 본인 명의로 통장이 발견돼서 연락을 드렸거든요.
>> scam call probability: 96.53355479240417% 

None
최근에 길고양이가 유기 센터에 들어오게 돼서 30일도 안 된 아기고양이를 우리 집에 데려온 적이 있어.
>> scam call probability: 2.6772325858473778% 

None


In [ ]:
# Code for checking the accuracy, precision, and recall
tp = []
fp = []
tn = []
fn = []
for d in dataset_test:
  result = predict(d[0])
  if result == 1 and int(d[1]) == 1:
    tp.append(d)
  elif result == 1 and int(d[1]) == 0:
    fp.append(d)
  elif result == 0 and int(d[1]) == 0:
    tn.append(d)
  elif result == 0 and int(d[1]) == 1:
    fn.append(d)


print(len(tp), len(fp), len(tn), len(fn))

이렇게 그이 사건에 대해서 한번 수사관은 개인 정보 보호법에 의거하여 혹시 주민번호, 은행권의 계좌번호, 비밀번호 등을 절대 묻지 않는다는 알아두시고 마찬가지로 시도 저희가 녹취 중에는 그 세 가지 절대 저한테 발사를 하시면 안 됩니다. 네 그럼 본인께서도 용의자에 일당들과 연관성이 없다는 것에 대해서 입점 조사 시작하겠습니다네. 저는 서울중앙지검 첨단 범죄수사 1부에 정제훈 수사관입니다. 본인 상황과 생년월일 말씀해 주세요. 이렇게 예금자 보호 등록을 위해 질문 드리겠습니다. 함께해서 운영하고 계신 자금이 합법적이고 이번 사건의 이성민 일단가 공무원 사실이 없는네 무고한 피해자라는 신분을 해명 받으셔야지만 금전적으로나 재산적으로 본인 앞으로 피해가 가지 않도록 저희가 안전 조치를 해 드릴 수가 있거든요. 그래서 역시 오늘 기준이에요. 어 현재 오늘 기준으로 본인의 통장에 얼마라고 기재가 되어 있어야 본인의 정상적이고 합법적인 제사입니까? 현재 진술하신 계좌상의 증명에 큰 오차가 날 경우 그 계좌 역시 정상 계좌가 아니라고 변조하고 본인 앞으로 피해 방지, 차 종결처리 및 어 체포 함장이 발부될 수가 있습니다. 약간 민감한 부분들이 제가 간단하게 설명을 해 드릴게요. 예를 들어서 입출이고 농협 입출금 통장에 저한테 100만 원이 있다고 말씀을 하셨어요. 저희가 계좌 수정을 했는데 조사 결과 100만 원이 아닌 터무니없는 천만 원이 나와요. 그러면 저희가 어떻게 생각을 하겠습니까? 그렇죠 저희는이 농협 통장 역시 어 범죄에 사용된 통장이다라고 생각할 수밖에 없죠. 그런데 그 통장 역시 동결 처리를 할 거고요. 어 공문장을 발송할 거예요. 근무 중에 적혀 있는 날짜를 보시고 직접 출석하셔서 일대일 주사도 따로 받으셔야 됩니다. 자 조사자 씨는 어 경기도 출신 38세 5명철이라는 사람 알고 계십니까? 전혀 모르시고요. 2017년 3월경에 경기도 광명시 철산 지점에서 하나은행과 우리은행 통장이 발급됐는데 보냈습니다. 직접 가셔서 발급받는 거 맞습니까? 대포 통장 형식

In [ ]:
accuray = (len(tp)+len(tn))/(len(tp)+len(tn)+len(fp)+len(fn))
precision = len(tp)/(len(tp)+len(fp))
recall = len(tp)/(len(tp)+len(fn))

print(accuray, precision, recall)

In [ ]:
# Infinite loop for testing, Quit when typing 0
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

# 8. Explainable AI (Not Implemented Yet)

In [ ]:
from captum_tools_vocvis import *
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

In [ ]:
PAD_IND = tok.vocab.padding_token
PAD_IND = tok.convert_tokens_to_ids(PAD_IND)
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model,model.bert.embeddings)

In [ ]:
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = 64, pad=True, pair=False)
voc_label_dict_inverse={ele:label_cols.index(ele) for ele in label_cols}
voc_label_dict={label_cols.index(ele):ele for ele in label_cols}

In [ ]:
def forward_with_sigmoid_for_bert(input,valid_length,segment_ids):
    return torch.sigmoid(model(input,valid_length,segment_ids))

In [ ]:
def forward_for_bert(input,valid_length,segment_ids):
    return torch.nn.functional.softmax(model(input,valid_length,segment_ids),dim=1)

In [ ]:
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 64, label = 0, n_steps=10):
    # tokenizing
    seq_tokens=transform([sentence])
    indexed=torch.tensor(seq_tokens[0]).long()#.to(device)
    valid_length=torch.tensor(seq_tokens[1]).long().unsqueeze(0)
    segment_ids=torch.tensor(seq_tokens[2]).long().unsqueeze(0).to(device)
    sentence=[token for token in tok.sentencepiece(sentence)]
    

    with torch.no_grad():
        model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid_for_bert(input_indices,valid_length,segment_ids).detach().cpu().numpy().argmax().item()
    print(forward_with_sigmoid_for_bert(input_indices,valid_length,segment_ids))
    pred_ind = round(pred)
    
    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices,\
                                           n_steps=n_steps, return_convergence_delta=True,target=label,\
                                           additional_forward_args=(valid_length,segment_ids))

    print('pred: ', Label.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, sentence, pred, pred_ind, label, delta, vis_data_records_ig)


In [ ]:
def add_attributions_to_visualizer(attributions, input_text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            voc_label_dict[pred_ind], #Label.vocab.itos[pred_ind],
                            voc_label_dict[label], # Label.vocab.itos[label],
                            100, # Label.vocab.itos[1],
                            attributions.sum(),       
                            input_text,
                            delta))


In [ ]:
visualize_text(vis_data_records_ig)